In [24]:
import random
import numpy as np
import pandas as pd
import joblib as jlb
from PIL import Image
from collections import OrderedDict
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from os import listdir

data_train = pd.read_csv('./data/train_distance.csv', header=None).to_numpy()
data_test = pd.read_csv('./data/test_distance.csv', header=None).to_numpy()
label = pd.read_csv('./data/training-label.csv', header=None).to_numpy()

print(data_train.shape)
print(data_test.shape)
print(label.shape)

mac_hash_train = data_train[1:, 1]
sniff_train = data_train[1:, 2]
time_train = data_train[1:, 3]
dist_train = data_train[1:, 4]
print(len(dist_train))

mac_hash_test = data_test[1:, 1]
sniff_test = data_test[1:, 2]
time_test = data_test[1:, 3]
dist_test = data_test[1:, 4]
print(len(mac_hash_test))

label = label[1:, :]

time_list_train = list(time_train)
d_train = {'Datatime': time_list_train}
df_train = pd.DataFrame(data=d_train)
df_train = pd.to_datetime(df_train['Datatime'], format = "%Y-%m-%d %H:%M:%S")

time_list_test = list(time_test)
d_test = {'Datatime': time_list_test}
df_test = pd.DataFrame(data=d_test)
df_test = pd.to_datetime(df_test['Datatime'], format = "%Y-%m-%d %H:%M:%S")

(41641, 5)
(20420, 5)
(6960, 2)
41640
20419


In [25]:
time_train = []
for i in range(len(mac_hash_train)):
    time_train.append(float(df_train[i].hour*3600+df_train[i].minute*60+df_train[i].second))

time_test = []
for i in range(len(mac_hash_test)):
    time_test.append(float(df_test[i].hour*3600+df_test[i].minute*60+df_test[i].second))

time_train = np.array(time_train)
time_test = np.array(time_test)

# scaler = StandardScaler().fit(time_train.reshape(-1, 1))
# time_train = scaler.transform(time_train.reshape(-1, 1))
# time_train = np.ndarray.flatten(time_train)
# time_test = scaler.transform(time_test.reshape(-1, 1))
# time_test = np.ndarray.flatten(time_test)

# print(time_train)
# print(time_test)

In [26]:
dist_train = np.array(dist_train)
dist_test = np.array(dist_test)
dist_scaler = StandardScaler().fit(dist_train.reshape(-1, 1))

dist_train = dist_scaler.transform(dist_train.reshape(-1, 1))
dist_train = np.ndarray.flatten(dist_train)

dist_test = dist_scaler.transform(dist_test.reshape(-1, 1))
dist_test = np.ndarray.flatten(dist_test)

print(dist_train)
print(dist_test)

[-1.32159609  0.12620552 -0.55959524 ... -0.15845227 -0.51157807
 -1.32159609]
[-1.32159609 -0.78819549 -0.51157807 ...  0.86044227  0.58340603
 -1.32159609]


In [27]:
mh_list_train = list(OrderedDict.fromkeys(mac_hash_train))

l_train = []
for i in range(len(mh_list_train)):
#     l_train.insert(i, [[mh_list_train[i]]])
    l_train.append([])
    
for i in range(len(mac_hash_train)):
#     ll = [float(sniff_train[i]), float(df_train[i].day-6), time_train[i], dist_train[i], float(type_train[i])]
    ll = [float(sniff_train[i]), float(df_train[i].day-6), time_train[i]]
    l_train[mh_list_train.index(mac_hash_train[i])].append(ll)

# for i in range(len(mh_list_train)):
#     mean = 0
#     for j in range(len(l_train[i])-2):
#         mean += l_train[i][j+2][2] - l_train[i][j+1][2]
#         l_train[i][j+1][2] = l_train[i][j+2][2] - l_train[i][j+1][2]

#     l_train[i][len(l_train[i])-1][2] = mean / (len(l_train[i])-1)

num = 0
for i in range(len(l_train)):
    num = num + len(l_train[i])

print(len(l_train))
print(l_train[1])
print(num)

6959
[[3.0, 1.0, 59375.0], [2.0, 1.0, 59441.0], [4.0, 1.0, 59826.0], [8.0, 1.0, 59827.0], [6.0, 1.0, 59828.0], [10.0, 1.0, 59890.0], [11.0, 1.0, 63132.0], [5.0, 2.0, 63038.0]]
41640


In [28]:
label_list = list(label[:, 1])

l2 = []
for i in range(len(mh_list_train)):
    l2.insert(i, [mh_list_train[i]])
    
for i in range(len(label)):
    target = int(label_list[i])
    for j in range(5):
        if target == j:
            k = 1.
        else:
            k = 0.
        l2[mh_list_train.index(label[i][0])].append(k)

print(len(l2))
dict2 = {}
for i in range(len(label)):
    dict2[i] = l2[i]

# print(dict2)
# jlb.dump(dict2, 'train_label.pkl')

# print(dict2)

6959


In [29]:
mh_list_test = list(OrderedDict.fromkeys(mac_hash_test))

l_test = []
for i in range(len(mh_list_test)):
#     l_test.insert(i, [[mh_list_test[i]]])
    l_test.append([])
    
for i in range(len(mac_hash_test)):
#     ll = [float(sniff_test[i]), float(df_test[i].day-6), time_test[i], dist_test[i], float(type_test[i])]
    ll = [float(sniff_test[i]), float(df_test[i].day-6), time_test[i]]
    l_test[mh_list_test.index(mac_hash_test[i])].append(ll)

num = 0
for i in range(len(l_test)):
    num = num + len(l_test[i])

print(len(l_test))
print(l_test[1])
print(num)

3429
[[10.0, 0.0, 49511.0], [9.0, 0.0, 49511.0], [11.0, 0.0, 49575.0], [8.0, 0.0, 49575.0], [12.0, 0.0, 50724.0], [4.0, 0.0, 58568.0], [7.0, 0.0, 58671.0], [2.0, 0.0, 58671.0], [6.0, 0.0, 58751.0], [13.0, 0.0, 60740.0]]
20419


In [30]:
d_train = np.zeros((len(l_train), 14))
d_train_d = np.zeros((len(l_train), 3))

for i in range(len(l_train)):
    for j in range(len(l_train[i])):
        d_train[i][int(l_train[i][j][0])-1] = 1
        d_train_d[i][int(l_train[i][j][1])-1] = 1
#         if l_train[i][j][1] == l_train[i][j+1][1]:
#             diff = l_train[i][j+1][2] - l_train[i][j][2]
#             if diff < 0:
#                 temp = l_train[i][j+1]
#                 l_train[i][j+1] = l_train[i][j]
#                 l_train[i][j] = temp
#                 diff *= -1
#             if diff >= 0 and diff < 1:
#                 d_train[i][int(l_train[i][j][0])-1] = 1
#             elif diff >= 1 and diff < 19:
#                 d_train[i][int(l_train[i][j][0])-1] = 2
#             elif diff >= 19 and diff < 120:
#                 d_train[i][int(l_train[i][j][0])-1] = 3
#             elif diff >= 120 and diff < 428:
#                 d_train[i][int(l_train[i][j][0])-1] = 4
#             elif diff >= 428:
#                 d_train[i][int(l_train[i][j][0])-1] = 5
#         else:
#             d_train[i][int(l_train[i][j][0])-1] = 1
#     d_train[i][int(l_train[i][len(l_train[i])-1][0])-1] = 1

    
d_test = np.zeros((len(l_test), 14))
d_test_d = np.zeros((len(l_test), 3))

for i in range(len(l_test)):
    for j in range(len(l_test[i])):
        d_test[i][int(l_test[i][j][0])-1] = 1
        d_test_d[i][int(l_test[i][j][1])-1] = 1
#         if l_test[i][j][1] == l_test[i][j+1][1]:
#             diff = l_test[i][j+1][2] - l_test[i][j][2]
#             if diff < 0:
#                 temp = l_test[i][j+1]
#                 l_test[i][j+1] = l_test[i][j]
#                 l_test[i][j] = temp
#                 diff *= -1
#             if diff >= 0 and diff < 1:
#                 d_test[i][int(l_test[i][j][0])-1] = 1
#             elif diff >= 1 and diff < 19:
#                 d_test[i][int(l_test[i][j][0])-1] = 2
#             elif diff >= 19 and diff < 120:
#                 d_test[i][int(l_test[i][j][0])-1] = 3
#             elif diff >= 120 and diff < 428:
#                 d_test[i][int(l_test[i][j][0])-1] = 4
#             elif diff >= 428:
#                 d_test[i][int(l_test[i][j][0])-1] = 5
#         else:
#             d_test[i][int(l_test[i][j][0])-1] = 1
#     d_test[i][int(l_test[i][len(l_test[i])-1][0])-1] = 1

In [31]:
df1 = pd.DataFrame(data=mh_list_train, columns=['mac_hash'])
label_mac = list(label[:, 0])
d_label = []
for i in range(len(l_train)):
    idx = label_mac.index(mh_list_train[i])
    d_label.append([mh_list_train[i], label_list[idx]])

df_label = pd.DataFrame(data=d_label, columns=['mac_hash', 'label'])
df_train = pd.DataFrame(data=d_train, columns=['sniff_loca01', 'sniff_loca02', 'sniff_loca03', 'sniff_loca04', 'sniff_loca05', 
                                               'sniff_loca06', 'sniff_loca07', 'sniff_loca08', 'sniff_loca09', 'sniff_loca10', 
                                               'sniff_loca11', 'sniff_loca12', 'sniff_loca13', 'sniff_loca14'])
df_train_d = pd.DataFrame(data=d_train_d, columns=['day1', 'day2', 'day3'])

df_train = pd.concat([df_label, df_train, df_train_d], axis=1)
df_train.to_csv('train_feature.csv')
df_train

,mac_hash,label,sniff_loca01,sniff_loca02,sniff_loca03,sniff_loca04,sniff_loca05,sniff_loca06,sniff_loca07,sniff_loca08,sniff_loca09,sniff_loca10,sniff_loca11,sniff_loca12,sniff_loca13,sniff_loca14,day1,day2,day3
0,0003ae8541d0e925fcee242287e2ad27,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1,00078611037990f7f36b722f22595fe7,1,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
2,000ce31739b333d931813de403156844,3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0
3,0027240b2aeb1f7f3c053c073cafb452,4,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
4,0028a89964416dc9a8a66cc7fe25ef4e,4,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6954,ffca215be2cb1505d931bd162da60104,4,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
6955,ffcaf0f29c6bede7b5cd9bc807c31715,1,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6956,ffd0c4403c3c44dcbe4772821b032252,4,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
6957,ffd7be58ee3c3fb47530ae7e4f251f88,0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [32]:
dict_train = {}

for i in range(len(l_train)):
    l = [[d_label[i][0]], list(d_train[i])]
    dict_train[i] = l

jlb.dump(dict_train, 'train_feature_location.pkl')
# print(dict_train)

['train_feature_location.pkl']

In [33]:
df2 = pd.DataFrame(data=mh_list_test, columns=['mac_hash'])
df_test = pd.DataFrame(data=d_test, columns=['sniff_loca01', 'sniff_loca02', 'sniff_loca03', 'sniff_loca04', 'sniff_loca05', 
                                             'sniff_loca06', 'sniff_loca07', 'sniff_loca08', 'sniff_loca09', 'sniff_loca10', 
                                             'sniff_loca11', 'sniff_loca12', 'sniff_loca13', 'sniff_loca14'])
df_test_d = pd.DataFrame(data=d_test_d, columns=['day1', 'day2', 'day3'])

df_test = pd.concat([df2, df_test, df_test_d], axis=1)
df_test.to_csv('test_feature.csv')
df_test

,mac_hash,sniff_loca01,sniff_loca02,sniff_loca03,sniff_loca04,sniff_loca05,sniff_loca06,sniff_loca07,sniff_loca08,sniff_loca09,sniff_loca10,sniff_loca11,sniff_loca12,sniff_loca13,sniff_loca14,day1,day2,day3
0,00c41469578fb947de736d3a03900772,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0104474d9c1318b4f58a75fb605f39b2,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0
2,01301359ac30dd5967c328f58282f13c,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,01911600eaa3baca34cf3e2f10ca353f,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
4,01ca752c7508d63665a2f62725286034,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3424,fed011e18897bfebe6b1d06288d9ed8d,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
3425,fed9dd416420f3dfe6104b3ada5798e6,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3426,feec5b8074464286c77c98096b8e5265,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3427,ffb29a79ebd3034876cc20c3d553dda6,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0


In [34]:
dict_test = {}

for i in range(len(l_test)):
    l = [[mh_list_test[i]], list(d_test[i])]
    dict_test[i] = l

jlb.dump(dict_test, 'test_feature_location.pkl')
# print(dict_test)

['test_feature_location.pkl']